### Data Ingestion


In [27]:
### document data structure
from langchain_core.documents import Document

In [28]:
doc=Document(
    page_content= "this is the main text content I am using to  create RAG",
    metadata={
        "source" : "example.txt",
        "pages": 1,
        "author" : "Harshiddhi Pathak",
        "date_created": "2025-11-25"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Harshiddhi Pathak', 'date_created': '2025-11-25'}, page_content='this is the main text content I am using to  create RAG')

In [29]:
##create a simple txt file
import os
os.makedirs("../data/text_files",exist_ok=True)

In [30]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction
    Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


"""
}

for filepath,content in sample_texts.items():
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(content)

print("sample txt file created!")

sample txt file created!


In [31]:
### TextLoader
from langchain_community.document_loaders import TextLoader
loader =TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document = loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n    Python is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [32]:
### directory loader

from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt", ## Pattern to match files  
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False

)

documents=dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n'),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n    Python is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popula

In [33]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/literature",
    glob="**/*.pdf", ## Pattern to match files  
    loader_cls= PyMuPDFLoader, ##loader class to use
    show_progress=False

)

pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': '', 'creator': '', 'creationdate': '2007-10-15T13:03:01+05:30', 'source': '..\\data\\literature\\A_Study_of_Combination_fMRI_and_DTI_in_A_Cognitive_Task.pdf', 'file_path': '..\\data\\literature\\A_Study_of_Combination_fMRI_and_DTI_in_A_Cognitive_Task.pdf', 'total_pages': 5, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20071105111301-05'", 'trapped': '', 'modDate': "D:20071105111301-05'", 'creationDate': "D:20071015130301+05'30'", 'page': 0}, page_content="2007 IEEE/ICME International Conference on Complex Medical En2ineerin2\nA Study of Combination fMRI and DTI in A\nCognitive Task\nTingting Wang', Zifan Cai1, Ni Tang1, Jie Lu3, Li Yao1 2 Xiaojie Zhaol2*\n1 School of Information Science and Technology, Beijing Normal University\n2 State Key Laboratory of Cognitive Neuroscience and Learning, Beijing Normal University\n3 Department of Radiology, Xuanwu Hospital of Capital University of Medical Sciences\n*zhao

In [34]:
type(pdf_documents[0])

langchain_core.documents.base.Document

In [35]:

import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [36]:

#### Embedding and vectorStoreDB

class EmbeddingManager:
    """ Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """

        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e} ")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
          
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generate embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generate embeddings with shape: {embeddings.shape}")
        return embeddings
    

## initialize the embedding manager 
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [37]:
import os
import uuid
import chromadb
import numpy as np
from typing import List, Any


class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(
        self,
        collection_name: str = "pdf_documents",
        persist_directory: str = "../data/vector_store"
    ):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent directory
            os.makedirs(self.persist_directory, exist_ok=True)

            # Create persistent ChromaDB client
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )

            print(f"✅ Vector store initialized. Collection: {self.collection_name}")
            print(f"📄 Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"❌ Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and embeddings to the vector store

        Args:
            documents: List of LangChain Document objects
            embeddings: NumPy array of corresponding embeddings
        """

        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):

            # Unique document ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Metadata
            metadata = dict(doc.metadata) if doc.metadata else {}
            metadata["doc_index"] = i
            metadata["content_length"] = len(doc.page_content)
            metadatas.append(metadata)

            # Document text
            documents_text.append(doc.page_content)

            # Convert embedding to list
            embeddings_list.append(embedding.tolist())

        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )

            print(f"✅ Successfully added {len(documents)} documents")
            print(f"📊 Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"❌ Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore      


✅ Vector store initialized. Collection: pdf_documents
📄 Existing documents in collection: 146


In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
print("Import working ✅")


Import working ✅


In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Split documents into smaller chunks for better RAG performance
    """

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    split_docs = text_splitter.split_documents(documents)

    print(f"✅ Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print("\n📌 Example chunk:")
        print(f"Metadata: {split_docs[0].metadata}")
        print(f"Content: {split_docs[0].page_content[:200]}...")

    return split_docs


In [40]:
chunks = split_documents(pdf_documents)
chunks

✅ Split 23 documents into 146 chunks

📌 Example chunk:
Metadata: {'producer': '', 'creator': '', 'creationdate': '2007-10-15T13:03:01+05:30', 'source': '..\\data\\literature\\A_Study_of_Combination_fMRI_and_DTI_in_A_Cognitive_Task.pdf', 'file_path': '..\\data\\literature\\A_Study_of_Combination_fMRI_and_DTI_in_A_Cognitive_Task.pdf', 'total_pages': 5, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20071105111301-05'", 'trapped': '', 'modDate': "D:20071105111301-05'", 'creationDate': "D:20071015130301+05'30'", 'page': 0}
Content: 2007 IEEE/ICME International Conference on Complex Medical En2ineerin2
A Study of Combination fMRI and DTI in A
Cognitive Task
Tingting Wang', Zifan Cai1, Ni Tang1, Jie Lu3, Li Yao1 2 Xiaojie Zhaol2*
...


[Document(metadata={'producer': '', 'creator': '', 'creationdate': '2007-10-15T13:03:01+05:30', 'source': '..\\data\\literature\\A_Study_of_Combination_fMRI_and_DTI_in_A_Cognitive_Task.pdf', 'file_path': '..\\data\\literature\\A_Study_of_Combination_fMRI_and_DTI_in_A_Cognitive_Task.pdf', 'total_pages': 5, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20071105111301-05'", 'trapped': '', 'modDate': "D:20071105111301-05'", 'creationDate': "D:20071015130301+05'30'", 'page': 0}, page_content="2007 IEEE/ICME International Conference on Complex Medical En2ineerin2\nA Study of Combination fMRI and DTI in A\nCognitive Task\nTingting Wang', Zifan Cai1, Ni Tang1, Jie Lu3, Li Yao1 2 Xiaojie Zhaol2*\n1 School of Information Science and Technology, Beijing Normal University\n2 State Key Laboratory of Cognitive Neuroscience and Learning, Beijing Normal University\n3 Department of Radiology, Xuanwu Hospital of Capital University of Medical Sciences\n*zhao

In [41]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generate embeddings for 146 texts...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches: 100%|██████████| 5/5 [00:18<00:00,  3.73s/it]


Generate embeddings with shape: (146, 384)
Adding 146 documents to vector store...
✅ Successfully added 146 documents
📊 Total documents in collection: 292


Retriever Pipeline From VectorStore

In [42]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
        # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [43]:
rag_retriever

In [44]:
rag_retriever.retrieve("What is attention is all you need")

Retrieving documents for query: 'What is attention is all you need'
Top K: 5, Score threshold: 0.0
Generate embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.04it/s]

Generate embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]

In [45]:
rag_retriever.retrieve("Unified Multi-task Learning Framework")

Retrieving documents for query: 'Unified Multi-task Learning Framework'
Top K: 5, Score threshold: 0.0
Generate embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.25it/s]

Generate embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]

RAG Pipeline- VectorDB To LLM Output Generation

In [57]:
import os
print(os.getenv("GROQ_API_KEY"))


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


In [58]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage



In [63]:
from dotenv import load_dotenv
import os

load_dotenv()   # loads .env file

api_key = os.getenv("GROQ_API_KEY")
print(api_key)  # should NOT be None


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


In [64]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [65]:
answer=rag_simple("What is attention mechanism?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is attention mechanism?'
Top K: 3, Score threshold: 0.0
Generate embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 28.48it/s]

Generate embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
No relevant context found to answer the question.


In [55]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
  

In [51]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [53]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("Unified Multi-task Learning Framework")

Retrieving documents for query: 'Unified Multi-task Learning Framework'
Top K: 5, Score threshold: 0.0
Generate embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 25.68it/s]

Generate embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]